In [171]:
import numpy as np
from Crypto.Util.number import getPrime, inverse
from hashlib import sha256
from random import randint
from Crypto.PublicKey import RSA
import math as m

# Generate a keypair for both the sender and receiver

In [172]:
# sender_keypair = RSA.generate(2048) #recommended 2048 key length
# receiver_keypair = RSA.generate(2048)

# pwd = b'secure_password'
# with open("sender_privkey.pem", "wb") as f:
#     data = sender_keypair.export_key(passphrase=pwd,
#                                 pkcs=8,
#                                 protection='PBKDF2WithHMAC-SHA512AndAES256-CBC',
#                                 prot_params={'iteration_count':131072})
#     f.write(data)

In [173]:
# pwd = b'secure_password_receiver'
# with open("receiver_privkey.pem", "wb") as f:
#     data = receiver_keypair.export_key(passphrase=pwd,
#                                 pkcs=8,
#                                 protection='PBKDF2WithHMAC-SHA512AndAES256-CBC',
#                                 prot_params={'iteration_count':131072})
#     f.write(data)

In [174]:
# with open("receiver_pubkey.pem", "wb") as f:
#     data = receiver_keypair.public_key().export_key()
#     f.write(data)

In [175]:
# with open("sender_pubkey.pem", "wb") as f:
#     data = sender_keypair.public_key().export_key()
#     f.write(data)

# Message (m) and Encryption -> M

In [176]:
msg = 'secret message'.encode('utf-8')



# Key generation

In [177]:
def generate_keypair():
    p = getPrime(1024)
    q = getPrime(1024)
    n = p * q
    phi_n = (p - 1) * (q - 1)

    while True:
        e = randint(2, phi_n - 1)
        if m.gcd(e, phi_n) == 1:
            break
    
    d = inverse(e, phi_n)

    public_key = (n, e)
    private_key = (n, d)

    return public_key, private_key

In [178]:
sender_keypair = generate_keypair()
receiver_keypair = generate_keypair()
print(sender_keypair, '\n', receiver_keypair)

((21864781890080711645946000083597301375250360276436529901087313737911439654918085877902280202971676267035879559773291793835120731922979144113465905941911156673088516856802398140738933013793034568258605571585317268381523721245445032043361099565543839668186043583610832980863214522126446026484132399637902148189437045202110842291114252135097291539138005412434331789693396439226303883112543328622626909228325773241855396631445493469366220411447930221345406647908521475066555586585209111114331230568084296604016224381772073475763855702688240194208412990473592851361546704044597194277115628305609901539373057477282542053909, 1540494103371610704008100322283658356562947862404551576669508992407248532347542247348427052709361432240305011009559996163426668851592757721712988836247267533493847308386752946034761755060579364107751582131322119076129649956893200207862558926227631394681877031090918170486603865255901937741704612307390180055360816887621654643880264864995975139958679715438942567545623056098429182

x -> encoded plaintext
y -> encrypted plaintext
h -> hashed x

# Encoding Message

In [179]:
msg = 'secret'.encode('utf-8')
x = int.from_bytes(msg, 'big')
x

126879297332596

In [180]:
hacker_msg = 'not secret'.encode('utf-8')
x_hack = int.from_bytes(hacker_msg, 'big')
x_hack

521516269522829685450100

# Encryption

In [181]:
receiver_pub, receiver_priv = receiver_keypair
n, e = receiver_pub
y = pow(x, e, n)
y

17215971576811706474270929259917946868569528507704487168517248037073774758852142919115299514060585154404094843622086968371181754840888093616417238904518771482777820532436926061465988464445262284671891582925964651025877291343055053127191549038539849313447422565634083769676028436335658218296198128352680979000381014844741140326190840654012163604510316508057733640650314016461938699993786367889418022391965799938834826001236167621368074564802314777493598599735371348487514179155005522998873011510005753879752508928741156456546496894237491839999903495412272921681454684967947969348419437074582857170268772711108411950863

In [182]:
y_hack = pow(x_hack, e, n)
y_hack

1516878359626374300651927458409832254502434899866667536702758754521322313569052599880443887215820335055420286065171313743747301251570741578279591001034986662480306728722935854498014092682214934913554130111000744457372583664206889825621661777612500928169795667848316659054279725556233261700935799087994320006005574721510063305924072874930196738381412139361773920568832916978979671577054249675161341933048992483222440413995811472176990211603380977198407214823604232001644247096675125360333998358403307379381903079755997298725751516181353330251724035882649813619060251346785710631883856902472635072604391900304559180385

# Hash

In [183]:
hash = sha256(msg)
h = int.from_bytes(hash.digest(),'big')

# Signing

In [184]:
sender_pub, sender_priv = sender_keypair
n, d = sender_priv
sign = pow(h, d, n)
sign 

8175093817914921482896040346325020663355474795342996364513255719100633021826200718120542525691169093792229479082821592034149942490863457528852328997337384944877810119899485886056467110709298347844499919370777140388785739660955833372265636081155100302927861985441208819015571302614940622251533739589881115955535259830993049915894855007001898304440997727142555110309266015811155621675607776926733563042563982708749326744686406411017531148940750081646072019158479378310126543591645034832455923552836901093803634856944147689983963991544079957688632438220513898747166238916284750903948769543790674704262615676140195033785

# Decryption of hash (sign) with Sender pub key

In [185]:
n, e = sender_pub
decrypt_to_hash = pow(sign, e, n)
decrypt_to_hash

19774644322343364210033507226347517504509547448996271814774638767344332546651

# Decryption of message

In [186]:
n, d = receiver_priv
decrypt_to_msg = pow(y, d, n)
decrypt_to_msg

126879297332596

# Hash verification 

In [187]:
msg_bytes = decrypt_to_msg.to_bytes((decrypt_to_msg.bit_length() + 7) // 8, 'big')
final_h = sha256(msg_bytes)
print(final_h.digest() == hash.digest())

True


# Get plaintext

In [188]:
decrypt_to_msg.to_bytes((decrypt_to_msg.bit_length() + 7) // 8, 'big').decode('utf-8')

'secret'